# Getting NFT Data

In [2]:
import pandas as pd
import numpy as np
import requests
import datetime
import time
import math

In [3]:
# Set the address or ENS name of the wallet to evaluate

name = 'dingaling.eth'

# call the getNFTs API
# filter out spam and airdrops 

key = 'MOtvi-7uep_FGJR9DPhWwmXOh1bfb3XH'

URL = 'https://eth-mainnet.g.alchemy.com/nft/v2/'+key+'/getNFTs?owner='+name+'&pageSize=100&withMetadata=false&excludeFilters[]=SPAM&excludeFilters[]=AIRDROPS'
r = requests.get(url = URL)

# if no data is returned we assume the wallet does not hold any NFTs

if r.status_code != 200:
    print ('No NFTs found')

In [4]:
# check out the json data shared by the API
# we get the contract, token id and quantity of every NFT in the waller

data = r.json()
data

{'ownedNfts': [{'contract': {'address': '0x00758fb0c83a68b6ce4fcf1d2665600f3f0e4570'},
   'id': {'tokenId': '0x00000000000000000000000000000000000000000000000000000000000000e3'},
   'balance': '1'},
  {'contract': {'address': '0x00758fb0c83a68b6ce4fcf1d2665600f3f0e4570'},
   'id': {'tokenId': '0x0000000000000000000000000000000000000000000000000000000000001b1c'},
   'balance': '1'},
  {'contract': {'address': '0x00758fb0c83a68b6ce4fcf1d2665600f3f0e4570'},
   'id': {'tokenId': '0x0000000000000000000000000000000000000000000000000000000000001b1b'},
   'balance': '1'},
  {'contract': {'address': '0x00758fb0c83a68b6ce4fcf1d2665600f3f0e4570'},
   'id': {'tokenId': '0x0000000000000000000000000000000000000000000000000000000000001b1a'},
   'balance': '1'},
  {'contract': {'address': '0x00758fb0c83a68b6ce4fcf1d2665600f3f0e4570'},
   'id': {'tokenId': '0x0000000000000000000000000000000000000000000000000000000000001ac2'},
   'balance': '1'},
  {'contract': {'address': '0x00758fb0c83a68b6ce4fcf1d266

In [5]:
# Using loop when pagekey is present

if 'pageKey' in data:
    
    more_pages = True
    holdings = pd.DataFrame(data['ownedNfts'])
    
    while more_pages:
        URL = 'https://eth-mainnet.g.alchemy.com/nft/v2/MOtvi-7uep_FGJR9DPhWwmXOh1bfb3XH/getNFTs?owner='+name+'&pageSize=100&withMetadata=false&excludeFilters[]=SPAM&excludeFilters[]=AIRDROPS&pageKey='+data['pageKey']
        r = requests.get(url = URL)
        data = r.json() 
        new_page = pd.DataFrame(data['ownedNfts'])
        holdings = holdings.append(new_page)
        if 'pageKey' not in data:
            more_pages = False
    
else:
    holdings = pd.DataFrame(data['ownedNfts'])

In [6]:
# Checking the dataframe

holdings.head()

,contract,id,balance
0,{'address': '0x00758fb0c83a68b6ce4fcf1d2665600...,{'tokenId': '0x0000000000000000000000000000000...,1
1,{'address': '0x00758fb0c83a68b6ce4fcf1d2665600...,{'tokenId': '0x0000000000000000000000000000000...,1
2,{'address': '0x00758fb0c83a68b6ce4fcf1d2665600...,{'tokenId': '0x0000000000000000000000000000000...,1
3,{'address': '0x00758fb0c83a68b6ce4fcf1d2665600...,{'tokenId': '0x0000000000000000000000000000000...,1
4,{'address': '0x00758fb0c83a68b6ce4fcf1d2665600...,{'tokenId': '0x0000000000000000000000000000000...,1


In [7]:
holdings.dtypes

contract    object
id          object
balance     object
dtype: object

In [8]:
# change the data type of the balance column to numeric
holdings['balance'] = pd.to_numeric(holdings['balance'])

# make a new column called address that takes the raw address out of the contract data 
holdings['address'] = holdings.apply(lambda x: x['contract']['address'],axis=1)

# trim the table to only include the address and balance columns
holdings = holdings[['address','balance']]

# sum up the balance of NFTs held by contract address
holdings = holdings.groupby(['address']).sum().reset_index()

In [11]:
holdings.head()

,address,balance
0,0x00758fb0c83a68b6ce4fcf1d2665600f3f0e4570,49
1,0x0100074e967602b2dac2e3810684517ed045af53,1
2,0x02d48fef34781cb84497b88ff46820b06331ce32,1
3,0x0477a628bd5722f56646b094126d4489b121b5ea,1
4,0x0574c34385b039c2bb8db898f61b7767024a9449,3


In [10]:
holdings

,address,balance
0,0x00758fb0c83a68b6ce4fcf1d2665600f3f0e4570,49
1,0x0100074e967602b2dac2e3810684517ed045af53,1
2,0x02d48fef34781cb84497b88ff46820b06331ce32,1
3,0x0477a628bd5722f56646b094126d4489b121b5ea,1
4,0x0574c34385b039c2bb8db898f61b7767024a9449,3
...,...,...
308,0xff72f37aa4eae3b7e1752e25db85b209f12c1a33,2
309,0xff8c5aeefbde324588528506c8b3e44da33c4630,1
310,0xff9c1b15b16263c61d017ee9f65c50e4ae0113d7,1
311,0xffc6dbff68a8e92a7984e474f7b7a9856945e0fb,1


# Get NFT Metadata

In [12]:
# Using getNFTMetadataBatch to fetch metadata for up to 100 NFT contracts a time

if len(holdings) < 100:
    
    contract_list = holdings['address'].tolist()
    url = "https://eth-mainnet.g.alchemy.com/nft/v2/MOtvi-7uep_FGJR9DPhWwmXOh1bfb3XH/getContractMetadataBatch"
    payload = {"contractAddresses": contract_list}
    headers = {
        "accept": "application/json",
        "content-type": "application/json"
    }

    r = requests.post(url, json=payload, headers=headers)
    data = r.json()
    metadata = pd.DataFrame(data)
    
else:

    # if the number of collections the wallet holds is more than 100, then the loop is used to pull the metadata in sets of 100

    # find how many sets of 100 there are in this group of contracts 
    number_of_pages = math.ceil(len(holdings)/100)

    # split the dataset into sets of 100
    holdings_list = np.array_split(holdings, number_of_pages)
    metadata_list = []
    
    # loop through the sets of 100
    # get each set's metadata and add it to a list
    for i in holdings_list:
        contract_list = i['address'].tolist()
        url = "https://eth-mainnet.g.alchemy.com/nft/v2/MOtvi-7uep_FGJR9DPhWwmXOh1bfb3XH/getContractMetadataBatch"
        payload = {"contractAddresses": contract_list}
        headers = {
            "accept": "application/json",
            "content-type": "application/json"
        }

        r = requests.post(url, json=payload, headers=headers)
        data = r.json()
        temp = pd.DataFrame(data)
        metadata_list.append(temp)
    
    # combine the sets of metadata into one table
    metadata = pd.concat(metadata_list, axis=0)

In [13]:
metadata.head()

,address,contractMetadata
0,0x00758fb0c83a68b6ce4fcf1d2665600f3f0e4570,"{'name': 'Juicebox Frens', 'symbol': 'JBF', 't..."
1,0x0100074e967602b2dac2e3810684517ed045af53,"{'name': 'CryptoSweepers Origin', 'symbol': 'C..."
2,0x02d48fef34781cb84497b88ff46820b06331ce32,"{'name': 'HadaPixels Original', 'symbol': 'Had..."
3,0x0477a628bd5722f56646b094126d4489b121b5ea,"{'name': 'SlicesOfTIMECovers', 'symbol': 'SOTC..."
4,0x0574c34385b039c2bb8db898f61b7767024a9449,"{'name': 'JOYWORLD JOYtoys', 'symbol': 'JOYtoy..."


In [14]:
# a function to extract OpenSea floorprice from the contractMetadata column of the table
def get_floorprice(x):
    try:
        return x['contractMetadata']['openSea']['floorPrice']
    except:
        return 0

# a function to extract the collection's name from the contractMetadata column of the table
def get_name(x):
    try:
        return x['contractMetadata']['name']
    except:
        return 'x'

# a function to extract the collection's image from the contractMetadata column of the table
def get_image(x):
    try:
        return x['contractMetadata']['openSea']['imageUrl']
    except:
        return 'x'

In [15]:
# create new column's in the metadata table for name, image and floor price
metadata['name'] = metadata.apply(lambda x: get_name(x),axis=1)
metadata['image'] = metadata.apply(lambda x: get_image(x),axis=1)
metadata['floor price'] = metadata.apply(lambda x: get_floorprice(x),axis=1)

In [16]:
# merge the holdings table (has contract address and balance columns) with the metadata we just made (has name, image, floor price for each nft contract)

portfolio = holdings.merge(metadata, how='inner', on='address')

In [17]:
# filter out collections that dont have a floor price
portfolio = portfolio[portfolio['floor price'] > 0]

# cut the portfolio table to only include relevant columns
portfolio = portfolio[['address', 'balance', 'name', 'image', 'floor price']]

# calculate the total value of each collection held by multiplying the target wallet's balance of each collection by its floor price
portfolio['value'] = portfolio['balance'] * portfolio['floor price']

In [18]:
portfolio.head()

,address,balance,name,image,floor price,value
0,0x00758fb0c83a68b6ce4fcf1d2665600f3f0e4570,49,Juicebox Frens,https://i.seadn.io/gae/PPkaPvl57hDacyDM_6MOprQ...,0.0042,0.2058
3,0x0477a628bd5722f56646b094126d4489b121b5ea,1,SlicesOfTIMECovers,https://i.seadn.io/gcs/files/4d91a72abeb8f27b8...,0.0100,0.0100
4,0x0574c34385b039c2bb8db898f61b7767024a9449,3,JOYWORLD JOYtoys,https://i.seadn.io/gae/tGu5kvHeUup-b65RY4YN49n...,0.0389,0.1167
5,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,4,Art Blocks,https://i.seadn.io/gcs/files/fd5e8fa6bb4e39cdd...,0.1200,0.4800
6,0x06012c8cf97bead5deae237070f9587f8e7a266d,5,CryptoKitties,https://i.seadn.io/gae/C272ZRW1RGGef9vKMePFSCe...,0.0039,0.0195


In [19]:
# calculate the total value of NFTs held by the target wallet
portfolio['value'].sum()

3237.9631611815485